In [6]:
import numpy as np
import json
import pandas as pd

### 配置来源

使用  precompute\data\1\1001 目录下的json文件作为数据源；然后整理出需要的numpy数组，以加速并行计算；
举例：
<!-- TODO: agent
这一部分准备在ipynb中一步步摸索，然后把案例整理到这里
-->
* 下面我们一步步的找数据，进行探索和整合：
* 对于 D:\fishinggame\ExportedData\Fishing_1006001_Dense_20260107_154037\Fishing_1006001_Global.npy，
先截取中其中的1006001，也就是scene id，然后到D:\fishinggame\precompute\data\1\1001\map_scene.json中，找到对应的map id

In [7]:
import re
from pathlib import Path

# 配置路径
DATA_ROOT = Path(r'D:\fishinggame\precompute\data\1\1001')
EXPORTED_DATA_ROOT = Path(r'D:\fishinggame\ExportedData')

# 加载 map_scene.json
with open(DATA_ROOT / 'map_scene.json', 'r', encoding='utf-8') as f:
    map_scene = json.load(f)

# 建立 assetId -> map_id 的反向索引
asset_to_map = {info['assetId']: int(map_id) for map_id, info in map_scene.items() if info.get('assetId')}
print(f'已加载 {len(map_scene)} 个地图配置，其中 {len(asset_to_map)} 个有 assetId')
print(f'assetId -> map_id 映射: {asset_to_map}')

已加载 9 个地图配置，其中 6 个有 assetId
assetId -> map_id 映射: {'1001001': 1001, '1002001': 1002, '1003001': 1003, '1004001': 1004, '1004002': 1008, '1006001': 1009}


In [8]:
def get_scene_id_from_path(npy_path: str) -> str:
    """从npy文件路径中提取scene_id (如 Fishing_1006001_Global.npy -> '1006001')"""
    match = re.search(r'Fishing_(\d+)', str(npy_path))
    if not match:
        raise ValueError(f'无法从路径中提取scene_id: {npy_path}')
    return match.group(1)

def get_map_id_from_scene_id(scene_id: str) -> int:
    """根据scene_id查找对应的map_id"""
    if scene_id in asset_to_map:
        return asset_to_map[scene_id]
    raise ValueError(f'找不到scene_id {scene_id} 对应的map_id')

def get_map_id_from_npy_path(npy_path: str) -> int:
    """从npy文件路径直接获取map_id"""
    scene_id = get_scene_id_from_path(npy_path)
    return get_map_id_from_scene_id(scene_id)

# 测试示例
test_path = r'D:\fishinggame\ExportedData\Fishing_1006001_Dense_20260107_154037\Fishing_1006001_Global.npy'
scene_id = get_scene_id_from_path(test_path)
map_id = get_map_id_from_scene_id(scene_id)
print(f'文件路径: {test_path}')
print(f'提取的 scene_id: {scene_id}')
print(f'对应的 map_id: {map_id}')
print(f'地图信息: {map_scene[str(map_id)]}')

文件路径: D:\fishinggame\ExportedData\Fishing_1006001_Dense_20260107_154037\Fishing_1006001_Global.npy
提取的 scene_id: 1006001
对应的 map_id: 1009
地图信息: {'id': 1009, 'name': 'map_base_6', 'desc': 106, 'assetId': '1006001', 'originOffsetX': 0, 'originOffsetY': 0, 'offsetX': 0, 'offsetY': 0, 'sizeX': 1000, 'sizeY': 1000, 'rotate': 0, 'mark': ''}


#### 对于局部池取stock
* 
* 去D:\fishinggame\precompute\data\1\1001\fish_stock.json当中，

In [9]:
# 加载额外的配置表
with open(DATA_ROOT / 'fish_pond_list.json', 'r', encoding='utf-8') as f:
    fish_pond_list = json.load(f)

with open(DATA_ROOT / 'fish_stock.json', 'r', encoding='utf-8') as f:
    fish_stock_config = json.load(f)

print(f"已加载 {len(fish_pond_list)} 个鱼塘配置")
print(f"已加载 {len(fish_stock_config)} 个 Stock 配置")

# 获取 map_id 对应的 map_scene 配置
current_map_info = map_scene.get(str(map_id))
if not current_map_info:
    raise ValueError(f"Found no map info for id {map_id}")

map_desc_id = current_map_info.get('desc')
print(f"\n当前地图: {current_map_info['name']} (ID: {map_id})")
print(f"关联的 Desc ID (用于对应 fish_pond_list.mapId): {map_desc_id}")

# 查找关联的 Pond 和 Stock
print(f"\n查找 mapId == {map_desc_id} 的鱼塘...")
related_ponds = [pond for pond in fish_pond_list.values() if pond.get('mapId') == map_desc_id]

if not related_ponds:
    print("警告: 未找到关联的鱼塘配置 (Pond)")
else:
    print(f"找到 {len(related_ponds)} 个关联鱼塘:")
    for pond in related_ponds:
        # 兼容处理: 有些json key可能是str类型的id
        pond_id = pond.get('id')
        pond_name = pond.get('name')
        stock_id = pond.get('fishStockId')
        
        print(f"  - Pond: {pond_name} (ID: {pond_id}) -> Stock ID: {stock_id}")
        
        # 查询 Stock 详情 (注意 key 可能是字符串)
        stock_info = fish_stock_config.get(str(stock_id))
        if stock_info:
             print(f"    Stock 详情: Name={stock_info.get('name')}, ResetTime={stock_info.get('resetDayTime')}")
        else:
             print(f"    警告: 在 fish_stock.json 中未找到 Stock ID {stock_id}")

已加载 7 个鱼塘配置
已加载 6 个 Stock 配置

当前地图: map_base_6 (ID: 1009)
关联的 Desc ID (用于对应 fish_pond_list.mapId): 106

查找 mapId == 106 的鱼塘...
找到 1 个关联鱼塘:
  - Pond: Sunset_Stream (ID: 301020005) -> Stock ID: 301030106
    Stock 详情: Name=stock_sunset, ResetTime=05:00


In [ ]:

# 顺着往下进行数据查找和组装numpy，供后面的计算使用。
# 大致思路为：
# 1. 用stock id，查询release id
# 2. with release id, find fishes configs within this stock (fish quality id, random weight, min len, max len, min weight, max weight, min env coff, min adapt coeff)
# 3. make fishes configs within this stock a pandas dataframe, we call it stockFishesPd, will use later.
# 4. use each fish quality id, find fish species
# 5. and fish env affinity ids.

In [10]:
import pandas as pd

# Load additional configurations
print("Loading Release and Quality configs...")
with open(DATA_ROOT / 'stock_release.json', 'r', encoding='utf-8') as f:
    stock_release_config = json.load(f)

with open(DATA_ROOT / 'fish_release.json', 'r', encoding='utf-8') as f:
    fish_release_config = json.load(f)

with open(DATA_ROOT / 'basic_fish_quality.json', 'r', encoding='utf-8') as f:
    basic_fish_quality_config = json.load(f)
print("Configs loaded.")

rows = []

# 'related_ponds' should be available from the previous cell execution
# If not, we rely on the logic that this cell is run after Cell 5.
if 'related_ponds' not in locals():
    print("Warning: 'related_ponds' not found. Please ensure the previous cell is executed.")
    unique_stock_ids = set()
else:
    unique_stock_ids = set(pond.get('fishStockId') for pond in related_ponds if pond.get('fishStockId'))

print(f"Processing {len(unique_stock_ids)} unique Stock IDs associated with the current map.")

for stock_id in unique_stock_ids:
    # Find all releases for this stock
    # Note: Scanning all values in stock_release_config might be inefficient for very large datasets,
    # but acceptable for this precompute scope.
    stock_releases = [item for item in stock_release_config.values() if item.get('stockId') == stock_id]
    
    for sr in stock_releases:
        release_id = sr.get('releaseId')
        fish_id = sr.get('fishId')
        fish_env_id = sr.get('fishEnvId')
        
        # Lookup Release Info
        release_info = fish_release_config.get(str(release_id))
        if not release_info:
            # print(f"Warning: Release ID {release_id} not found in fish_release.json")
            continue
            
        # Lookup Fish Quality Info
        fish_info = basic_fish_quality_config.get(str(fish_id))
        species_id = fish_info.get('species', -1) if fish_info else -1
        quality_id = fish_info.get('id', -1) if fish_info else -1 # Using ID as quality identifier for now
            
        row = {
            'stockId': stock_id,
            'releaseId': release_id,
            'fishId': fish_id,
            'envId': fish_env_id,
            'speciesId': species_id,
            'qualityId': quality_id,
            
            # Release Limits
            'weight_min': release_info.get('weightMin'),
            'weight_max': release_info.get('weightMax'),
            'len_min': release_info.get('lengthMin'),
            'len_max': release_info.get('lengthMax'),
            
            # Debug/Display info
            'name': release_info.get('name'),
            'probWeight': release_info.get('probWeightIdeal')
        }
        rows.append(row)

stockFishesPd = pd.DataFrame(rows)
print(f"Created stockFishesPd with {len(stockFishesPd)} rows.")
if not stockFishesPd.empty:
    print(stockFishesPd.head().to_string())
    print("\nColumn Types:")
    print(stockFishesPd.dtypes)
else:
    print("DataFrame is empty. Check if stock_release.json maps correctly to the pond stock IDs.")

Loading Release and Quality configs...
Configs loaded.
Processing 1 unique Stock IDs associated with the current map.
Created stockFishesPd with 35 rows.
     stockId  releaseId     fishId    envId  speciesId  qualityId  weight_min  weight_max  len_min  len_max                                name  probWeight
0  301030106     300500  101034430  1013390  101020063  101034430         150         450       26       37  Release_American_Shad_Young_sunset      250000
1  301030106     300510  101034090  1013050  101020010  101034090          50         200       16       26    Release_Brook_Trout_Young_sunset      100000
2  301030106     300520  101031007  1010066  101020010  101031007         200         350       26       32   Release_Brook_Trout_Common_sunset      100000
3  301030106     300530  101034450  1013410  101020003  101034450         150         450       28       40         Release_Bowfin_Young_sunset      200000
4  301030106     300540  101034510  1013470  101020050  101034510 